# Constructing Models

With a few exceptions, most models are constructed by describing the model in JSON format, and passing the JSON-formatted information to the ``make_model`` function.  There are some convenience functions exposed for backwards compatibility, but as of version 0.14.0, all model construction should go via this route.

At the C++ level, the returned value from the ``make_model`` function is a ``shared_ptr`` that wraps a pointer to an ``AbstractModel`` class.  The ``AbstractModel`` class is an abstract class which defines the public C++ interface.

In Python, construction is in two parts. First, the model is constructed, which only includes the common methods.  Then, the model-specific attributes and methods are attached with the ``attach_model_specific_methods`` method.

The JSON structure is of two parts, the ``kind`` field is a case-sensitive string defining which model kind is being constructed, and the ``model`` field contains all the information needed to build the model. In the case of hard-coded models, nothing is provided in the ``model`` field, but it must still be provided.

Also, the argument to ``make_model`` must be valid JSON. So if you are working with numpy array datatypes, make sure to convert them to a list (which is convertible to JSON). Example below.

In [1]:
import teqp, numpy as np
teqp.__version__

'0.17.0'

In [2]:
teqp.make_model({'kind': 'vdW1', 'model': {'a': 1, 'b': 2}})

In [3]:
# Fields are case-sensitive
teqp.make_model({'kind': 'vdW1', 'model': {'a': 1, 'B': 2}})

RuntimeError: :{"B":2,"a":1}': required property 'b' not found in object
|/|\|:{"B":2,"a":1}': validation failed for additional property 'B': instance invalid as per false-schema


In [4]:
# A hard-coded model
teqp.make_model({
    'kind': 'AmmoniaWaterTillnerRoth', 
    'model': {}
})

In [5]:
# Show what to do with numpy array
Tc_K = np.array([100,200])
pc_Pa = np.array([3e6, 4e6])
teqp.make_model({
    "kind": "vdW", 
    "model": {
        "Tcrit / K": Tc_K.tolist(), 
        "pcrit / Pa": pc_Pa.tolist()
    }
})

In [6]:
# methane with conventional PC-SAFT
j = {
    'kind': 'PCSAFT',
    'model': {
        'coeffs': [{
            'name': 'methane',
             'BibTeXKey': 'Gross-IECR-2001',
             'm': 1.00,
             'sigma_Angstrom': 3.7039,
             'epsilon_over_k': 150.03,
        }]
    }
}
model = teqp.make_model(j)